In [1]:
import pandas as pd
from analysis import *
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import binom_test

In [2]:
surge_annotations = data.surge_evaluation.annotation_dataframe()
surge_annotations_comparative = data.surge_evaluation.comparative_annotation_dataframe()

surge_annotations

0  \
bot            category        label        item                               
rerank_blender likert dialogue emotional    (109,13)_rerank_blender        4   
                               consistent   (109,13)_rerank_blender        4   
                               grammatical  (109,13)_rerank_blender        4   
                               informative  (109,13)_rerank_blender        4   
                               proactive    (109,13)_rerank_blender        4   
...                                                                       ..   
               behavior        follow up    ((438,26)_rerank_blender, 14)  0   
                               topic switch ((438,26)_rerank_blender, 14)  0   
                               ignore       ((438,26)_rerank_blender, 14)  0   
                               irrelevant   ((438,26)_rerank_blender, 14)  0   
                               antisocial   ((438,26)_rerank_blender, 14)  0   

                                                                             1  \
bot            category        label        item                                 
rerank_blender likert dialogue emotional    (109,13)_rerank_blender        4.0   
                               consistent   (109,13)_rerank_blender        4.0   
                               grammatical  (109,13)_rerank_blender        3.0   
                               informative  (109,13)_rerank_blender        4.0   
                               proactive    (109,13)_rerank_blender        3.0   
...                                                                        ...   
               behavior        follow up    ((438,26)_rerank_blender, 14)  NaN   
                               topic switch ((438,26)_rerank_blender, 14)  NaN   
                               ignore       ((438,26)_rerank_blender, 14)  NaN   
                               irrelevant   ((438,26)_rerank_blender, 14)  NaN   
                               antisocial   ((438,26)_rerank_blender, 14)  NaN   

                                                                            2  
bot            category        label        item                               
rerank_blender likert dialogue emotional    (109,13)_rerank_blender       NaN  
                               consistent   (109,13)_rerank_blender       NaN  
                               grammatical  (109,13)_rerank_blender       NaN  
                               informative  (109,13)_rerank_blender       NaN  
                               proactive    (109,13)_rerank_blender       NaN  
...                                                                        ..  
               behavior        follow up    ((438,26)_rerank_blender, 14) NaN  
                               topic switch ((438,26)_rerank_blender, 14) NaN  
                               ignore       ((438,26)_rerank_blender, 14) NaN  
                               irrelevant   ((438,26)_rerank_blender, 14) NaN  
                               antisocial   ((438,26)_rerank_blender, 14) NaN  

[151664 rows x 3 columns]

# 8 Comprehensive Analysis

### Metric Sensitivity

In [16]:
from itertools import combinations
from scipy.stats import ttest_ind

def p_vals(df: pd.DataFrame, test='t', downsample=None):
    """
    :param df: (bot, data point) x 1 -> score
    :param test: statistical test function (t for t test, p for prop test, s for sign test)
    :param downsample: number of samples ber bot to subsample without replacement for the analysis
    :return: p values of test on each bot pair (pd.Series)
    """
    seed = 123
    bots = set(df.index.get_level_values(0))
    num_bots = len(bots)
    bot_pairs = list(combinations(bots, 2))
    result = {}
    for ba, bb in bot_pairs:
        if test == 't':
            if downsample:
                a = df.xs(ba).sample(downsample, random_state=seed).to_numpy().squeeze()
                b = df.xs(bb).sample(downsample, random_state=seed).to_numpy().squeeze()
            else:
                a = df.xs(ba).to_numpy().squeeze()
                b = df.xs(bb).to_numpy().squeeze()
            t, p = ttest_ind(a, b, equal_var=False)
        elif test == 'p':
            if downsample:
                a = df.xs(ba).sample(downsample, random_state=seed).to_numpy().squeeze()
                b = df.xs(bb).sample(downsample, random_state=seed).to_numpy().squeeze()
            else:
                a = df.xs(ba).to_numpy().squeeze()
                b = df.xs(bb).to_numpy().squeeze()
            z, p = proportions_ztest(count=[
                sum(a), sum(b)
            ], nobs=[
                len(a), len(b)
            ])
        elif test == 's':
            # sign test
            comp_data = df.xs((ba, bb), level=[sym.bot, sym.bot_cmp])
            if downsample:
                comp_data = comp_data.sample(downsample, random_state=seed)
            a = comp_data.to_numpy().squeeze() == 1
            b = comp_data.to_numpy().squeeze() == -1
            p = binom_test(sum(a), sum(a)+sum(b), p=0.5)
        else:
            raise ValueError('invalid arg for param "test"')
        result[(ba, bb)] = p
    result_series = pd.Series(result.values(), result)
    return result_series

@to_file
def p_values_comparing_bots(evaluation, downsample=None):
    comp_annotations = get_singly_annotated(evaluation.comparative_annotation_dataframe(), seed=123)
    annotations = get_singly_annotated(evaluation.annotation_dataframe(), seed=123)
    prop_annotations = annotations.xs(
        category.behavior, level=sym.category, drop_level=False
    )
    mean_annotations = annotations.drop(
        index=category.behavior, level=sym.category
    ).drop(
        index=category.comparative, level=sym.category
    )
    mean_ps = mean_annotations.groupby(
        [sym.category, sym.label]
    ).apply(lambda x: p_vals(x, test='t', downsample=downsample))
    prop_ps = prop_annotations.groupby(
        [sym.category, sym.label]
    ).apply(lambda x: p_vals(x, test='p', downsample=downsample))
    comp_groups = comp_annotations.groupby(sym.label)
    comp_ps = comp_groups.apply(lambda x: p_vals(x, test='s', downsample=downsample))
    comp_ps = pd.concat({category.comparative: comp_ps}, names=[sym.category])
    result = pd.concat([prop_ps, mean_ps, comp_ps], axis=0)
    return result

p_values_comparing_bots(data.surge_evaluation, downsample=32, reload='results/p_values_comparing_bots_downsampled').round(4)

/home/james/anaconda3/envs/Research/lib/python3.10/site-packages/statsmodels/stats/weightstats.py:790: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std
/home/james/anaconda3/envs/Research/lib/python3.10/site-packages/statsmodels/stats/weightstats.py:790: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std
/home/james/anaconda3/envs/Research/lib/python3.10/site-packages/statsmodels/stats/weightstats.py:790: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std


bart_fid_rag_bcb              \
                                                     emora blender2_3B   
category        label                                                    
behavior        antisocial                             NaN         NaN   
                commonsense contradiction           0.1685      0.3202   
                correct fact                        0.0002      0.0098   
                empathetic                          0.6107      0.7978   
                follow up                           0.0002      0.5986   
                ignore                              0.4911      0.1623   
                incorrect fact                      0.0101      0.0101   
                irrelevant                          0.2002      0.6888   
                lack of empathy                     0.5218      0.2296   
                life info                           0.7404      0.5453   
                partner contradiction               0.6414      0.3017   
                preference info                     0.0016      0.3164   
                redundant                           0.6888      0.3911   
                self contradiction                  0.0101      0.0452   
                topic switch                        0.0079      0.1685   
                uninterpretable                        NaN         NaN   
likert dialogue consistent                          0.0243      0.1380   
                emotional                           0.2968      0.0002   
                engaging                            0.0968      0.0022   
                grammatical                         0.8801      0.1028   
                informative                         0.6013      0.7968   
                proactive                           0.0000      0.0000   
                quality                             0.3231      0.0005   
                relevant                            0.5454      0.0465   
likert turn     consistent                          0.4576      0.0804   
                emotional                           0.0173      0.1112   
                engaging                            0.2696      0.2934   
                grammatical                         0.0210      0.0012   
                informative                         0.0052      0.2691   
                proactive                           0.0013      0.0024   
                quality                             0.6389      0.0107   
                relevant                            0.8537      0.7245   
comparative     consistent                          0.0501      0.5966   
                emotional                           0.2005      0.5847   
                engaging                            0.1102      0.8601   
                grammatical                         0.5716      0.8388   
                informative                         0.2153      0.0201   
                proactive                           0.0107      0.1102   
                quality                             0.4731      0.3771   
                relevant                            0.0501      1.0000   

                                                               emora  \
                                          rerank_blender blender2_3B   
category        label                                                  
behavior        antisocial                           NaN         NaN   
                commonsense contradiction         0.5218      0.6888   
                correct fact                      0.0211      0.1685   
                empathetic                        0.2807      0.8002   
                follow up                         0.0005      0.0010   
                ignore                            0.4911      0.0452   
                incorrect fact                    0.4911         NaN   
                irrelevant                        0.3202      0.0976   
                lack of empathy                   0.2296      0.072

In [17]:
p_values_comparing_bots(data.surge_evaluation, reload='results/t_test_p_values_comparing_bots').round(4)

bart_fid_rag_bcb              \
                                                     emora blender2_3B   
category        label                                                    
behavior        antisocial                          0.5597      0.1601   
                commonsense contradiction           0.0000      0.0002   
                correct fact                        0.0000      0.0000   
                empathetic                          0.0000      0.0000   
                follow up                           0.0000      0.5851   
                ignore                              0.3671      0.0436   
                incorrect fact                      0.0000      0.0000   
                irrelevant                          0.0019      0.1866   
                lack of empathy                     0.6253      0.0000   
                life info                           0.0000      0.9618   
                partner contradiction               0.1876      0.9333   
                preference info                     0.0000      0.9816   
                redundant                           0.3665      0.0017   
                self contradiction                  0.0000      0.0000   
                topic switch                        0.0000      0.0002   
                uninterpretable                     0.0001      0.0016   
likert dialogue consistent                          0.0000      0.0138   
                emotional                           0.7603      0.0000   
                engaging                            0.0049      0.0000   
                grammatical                         0.1644      0.0005   
                informative                         0.0209      0.2968   
                proactive                           0.0000      0.0000   
                quality                             0.0003      0.0000   
                relevant                            0.5081      0.0009   
likert turn     consistent                          0.0002      0.0043   
                emotional                           0.2403      0.0000   
                engaging                            0.0000      0.0000   
                grammatical                         0.0000      0.0000   
                informative                         0.0000      0.0000   
                proactive                           0.0000      0.0000   
                quality                             0.0001      0.0000   
                relevant                            0.0693      0.0000   
comparative     consistent                          0.0501      0.5966   
                emotional                           0.2005      0.5847   
                engaging                            0.1102      0.8601   
                grammatical                         0.5716      0.8388   
                informative                         0.2153      0.0201   
                proactive                           0.0107      0.1102   
                quality                             0.4731      0.3771   
                relevant                            0.0501      1.0000   

                                                               emora  \
                                          rerank_blender blender2_3B   
category        label                                                  
behavior        antisocial                        0.0119      0.0587   
                commonsense contradiction         0.0473      0.0008   
                correct fact                      0.0000      0.0000   
                empathetic                        0.0000      0.5285   
                follow up                         0.0000      0.0000   
                ignore                            0.0079      0.0035   
                incorrect fact                    0.0663      0.0023   
                irrelevant                        0.0000      0.0000   
                lack of empathy                   0.0000      0.000

### Predictive Validity

In [49]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.regression.linear_model import OLS as LinearModel
from statsmodels.tools.tools import add_constant


def all_dialogue_metrics(data):
    static: pd.DataFrame = data.surge_evaluation.annotation_dataframe()
    static = get_singly_annotated(static, seed=123)
    reindexed = static.reset_index()
    items = reindexed[sym.item]
    dialogues = [e[0] if isinstance(e, tuple) else e for e in items]
    reindexed['dialogue'] = dialogues
    reindexed.set_index(
        [sym.bot, sym.category, sym.label, 'dialogue', sym.item],
        inplace=True, verify_integrity=True
    )
    ld = reindexed.xs(category.likert_dialogue, level=sym.category)
    ld = ld.droplevel(sym.bot).droplevel(sym.item)
    ld.columns = ['score']
    ldq = ld.xs(scale.quality, level=sym.label)
    ldq.columns = ['quality']

    lt = reindexed.xs(category.likert_turn, level=sym.category)
    lt = lt.groupby([sym.label, 'dialogue']).mean()
    lt.columns = ['score']
    ltq = lt.xs(scale.quality, level=sym.label)
    ltq.columns = ['quality']

    be = reindexed.xs(category.behavior, level=sym.category)
    be = be.groupby([sym.label, 'dialogue']).mean()
    be.columns = ['score']

    interactive = data.dialogue_collection.annotation_dataframe()
    idq = interactive.xs((category.likert_dialogue, scale.quality), level=(sym.category, sym.label))
    idq = idq.droplevel(0)

    ds = pd.concat(
        [lt, be, ld],
        keys=[category.likert_turn, category.behavior, category.likert_dialogue],
        names=[sym.category, sym.label, 'dialogue']
    )
    likert_dialogue_quality_features = ds.join(ldq, on='dialogue')
    likert_turn_quality_features = ds.join(ltq, on='dialogue')
    interactive_dialogue_quality_features = ds.join(idq, on='dialogue')
    interactive_dialogue_quality_features.columns = ['score', 'quality']

    interactive_comparisons = data.dialogue_collection.comparative_annotation_dataframe()
    surge_comparisons = get_singly_annotated(data.surge_evaluation.comparative_annotation_dataframe(), seed=123)
    compared_dialogues = surge_comparisons.index.get_level_values('dialogues')
    unique_compared_dialogues = {tuple(x) for x in {frozenset(y) for y in compared_dialogues}}
    comparison_map = dict(unique_compared_dialogues)
    compared_selector = [
        pair in unique_compared_dialogues
        for pair in interactive_comparisons.index.get_level_values('dialogues')
    ]
    comparative: pd.DataFrame = interactive_comparisons.loc[compared_selector, :]
    compared_selector = [
        pair in unique_compared_dialogues
        for pair in surge_comparisons.index.get_level_values('dialogues')
    ]
    surge_comparisons: pd.DataFrame = surge_comparisons.loc[compared_selector, :]
    comparative_quality = comparative.xs(scale.quality, level=sym.label)
    comparative_quality.index = [first for _, _, (first, second) in comparative_quality.index.values]
    comparative_quality.columns = ['quality']
    surge_comparisons.index = pd.MultiIndex.from_arrays(
        list(zip(*[
            (category.comparative, label, left)
            for _, _, label, (left, right) in surge_comparisons.index.values
        ])),
        names=[sym.category, sym.label, 'dialogue']
    )
    surge_comparisons.columns = ['score']
    filtered_ds = ds.loc[[(c, l, d) for c, l, d in ds.index.values if d in comparison_map]]
    compared_features = ds.loc[[(c, l, comparison_map[d]) for c, l, d in filtered_ds.index.values]]
    comparative_features = filtered_ds.to_numpy() - compared_features.to_numpy()
    filtered_ds['diff'] = comparative_features.squeeze().tolist()
    del filtered_ds['score']
    filtered_ds.columns = ['score']
    filtered_ds = pd.concat([filtered_ds, surge_comparisons], axis=0)
    comparative_quality_features = filtered_ds.join(comparative_quality, on='dialogue')

    return (
        likert_dialogue_quality_features,
        likert_turn_quality_features,
        comparative_quality_features,
        interactive_dialogue_quality_features
    )

all_dialogue_metrics(data)

def regressions(df, quality_column_name=None, model='linear'):
    """
    :param df: dialogue x (*features, quality) -> value
    :return: *(coef, low, high), mcfadden r^2
    """
    if not quality_column_name:
        quality_column_name = df.columns[-1]
    qualities = df[quality_column_name]
    features = [f for f in df.columns if f != quality_column_name]
    if model == 'ordinal':
        model = OrderedModel(qualities, df[features], distr='logit')
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        prsqrd = results.prsquared
        result = {stat.mcfad_r2: prsqrd, stat.p_of_llr_test: results.llr_pvalue}
    elif model == 'linear':
        x = add_constant(df[features])
        y = qualities
        model = LinearModel(y, x)
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        rsquared = results.rsquared
        result = {**coefs, stat.r2: rsquared, stat.p_of_f_test: results.f_pvalue}
    else:
        raise ValueError('Param "model" must be one of {"linear", "ordinal"}')
    return pd.Series(result.values(), result)



In [50]:
@to_file
def dialogue_quality_regressions(data):
    ldq, ltq, icq, idq = all_dialogue_metrics(data)
    ldq_groups = ldq.groupby([sym.category, sym.label])
    ltq_groups = ltq.groupby([sym.category, sym.label])
    icq_groups = icq.groupby([sym.category, sym.label])
    idq_groups = idq.groupby([sym.category, sym.label])
    names = ['Predicted', 'Metric']
    linear_result = ldq_groups.apply(lambda x: regressions(x, model='linear'))
    linear_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Dialogue Quality']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    ordinal_result = ldq_groups.apply(lambda x: regressions(x, model='ordinal'))
    ordinal_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Dialogue Quality']*2,
        ['OR Pseudo R-Squared', stat.p_of_llr_test]],
        names=names
    )
    linear_turn_result = ltq_groups.apply(regressions)
    linear_turn_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Turn Quality']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    linear_compare_result = icq_groups.apply(regressions)
    linear_compare_result.columns = pd.MultiIndex.from_arrays(
        [['Interactive Comparison']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    interactive_dial_result = idq_groups.apply(regressions)
    interactive_dial_result.columns = pd.MultiIndex.from_arrays(
        [['Interactive Likert']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    result = pd.concat(( linear_compare_result, interactive_dial_result, linear_result, linear_turn_result), axis=1)
    return result.round(5)

regs = dialogue_quality_regressions(
    data,
    load='results/dialogue_quality_regressions'
)
regs

Predicted                                 Interactive Comparison               \
Metric                                            LR Coefficient LR R-Squared   
category        label                                                           
behavior        antisocial                              -0.28946      0.00006   
                commonsense contradiction               -1.29356      0.06519   
                correct fact                            -0.51742      0.01660   
                empathetic                               0.46971      0.01802   
                follow up                                0.24425      0.00585   
                ignore                                  -0.95919      0.01765   
                incorrect fact                          -0.84437      0.03112   
                irrelevant                              -0.61683      0.01176   
                lack of empathy                         -1.37372      0.05880   
                life info                                0.41711      0.00985   
                partner contradiction                   -1.19561      0.02366   
                preference info                          0.17608      0.00244   
                redundant                               -1.43331      0.02485   
                self contradiction                      -1.88253      0.05467   
                topic switch                             0.27619      0.00218   
                uninterpretable                         -2.72176      0.01343   
comparative     consistent                               0.10866      0.01223   
                emotional                                0.17283      0.02900   
                engaging                                 0.18520      0.03525   
                grammatical                              0.12158      0.01315   
                informative                              0.01279      0.00017   
                proactive                                0.12632      0.01650   
                quality                                  0.15941      0.02621   
                relevant                                 0.09643      0.00947   
likert dialogue consistent                               0.09869      0.03437   
                emotional                                0.18336      0.07204   
                engaging                                 0.25383      0.14639   
                grammatical                              0.05884      0.00648   
                informative                              0.10078      0.01829   
                proactive                                0.09677      0.02640   
                quality                                  0.20634      0.07572   
                relevant                                 0.14551      0.05097   
likert turn     consistent                               0.25838      0.06045   
                emotional                                0.11313      0.01170   
                engaging                                 0.09135      0.01076   
                grammatical                              0.15085      0.01652   
                informative                             -0.20004      0.03776   
                proactive                                0.30850      0.12035   
                quality                                  0.25014      0.05242   
                relevant                                 0.15989      0.03294   

Predicted                                                    \
Metric                                    P value of F-test   
category        label                                         
behavior        antisocial                          0.91533   
                commonsense contradiction           0.00035   
                correct fact                        0.07488   
                empathetic                          0.06341   
                follow up                           0.29152   
            

In [51]:
to_plot_regs = regs[[("Likert Dialogue Quality", "LR R-Squared"), ("Likert Dialogue Quality", "P value of F-test")]]
to_plot_regs = to_plot_regs.drop(("likert dialogue", "quality"))
to_plot_regs = to_plot_regs.reset_index()
to_plot_regs

Predicted         category                      label Likert Dialogue Quality  \
Metric                                                           LR R-Squared   
0                 behavior                 antisocial                 0.00013   
1                 behavior  commonsense contradiction                 0.06103   
2                 behavior               correct fact                 0.00010   
3                 behavior                 empathetic                 0.04044   
4                 behavior                  follow up                 0.00440   
5                 behavior                     ignore                 0.03455   
6                 behavior             incorrect fact                 0.01915   
7                 behavior                 irrelevant                 0.04943   
8                 behavior            lack of empathy                 0.04384   
9                 behavior                  life info                 0.00306   
10                behavior      partner contradiction                 0.06043   
11                behavior            preference info                 0.00306   
12                behavior                  redundant                 0.02931   
13                behavior         self contradiction                 0.02745   
14                behavior               topic switch                 0.00001   
15                behavior            uninterpretable                 0.00772   
16             comparative                 consistent                     NaN   
17             comparative                  emotional                     NaN   
18             comparative                   engaging                     NaN   
19             comparative                grammatical                     NaN   
20             comparative                informative                     NaN   
21             comparative                  proactive                     NaN   
22             comparative                    quality                     NaN   
23             comparative                   relevant                     NaN   
24         likert dialogue                 consistent                 0.22558   
25         likert dialogue                  emotional                 0.23022   
26         likert dialogue                   engaging                 0.38103   
27         likert dialogue                grammatical                 0.07243   
28         likert dialogue                informative                 0.17475   
29         likert dialogue                  proactive                 0.27452   
30         likert dialogue                   relevant                 0.28720   
31             likert turn                 consistent                 0.02999   
32             likert turn                  emotional                 0.02943   
33             likert turn                   engaging                 0.02729   
34             likert turn                grammatical                 0.00010   
35             likert turn                informative                 0.00159   
36             likert turn                  proactive                 0.02868   
37             likert turn                    quality                 0.02906   
38             likert turn                   relevant                 0.03776   

Predicted                    
Metric    P value of F-test  
0                   0.82001  
1                   0.00000  
2                   0.84188  
3                   0.00005  
4                   0.18548  
5                   0.00019  
6                   0.00557  
7                   0.00001  
8                   0.00002  
9                   0.26971  
10                  0.00000  
11                  0.26993  
12                  0.00058  
13                  0.00088  
14                  0.94880  
15                  0.07922  
16                      NaN  
17                      NaN  
18                      NaN  
19                      NaN  
20                

In [52]:
regs = prettify(regs, to_csv="results/paper/predictive_validity.csv")

# Incremental Validity

In [53]:
def drop_column_level_duplication(df: pd.DataFrame, columns, levels=None):
    if levels is None:
        levels = list(range(len(columns)))
    level_columns = df.xs(columns, axis=1, level=levels)
    unique = level_columns.iloc[:,0].to_frame()
    unique.columns = [columns]
    dropped = df.drop(columns=columns, level=levels)
    result = pd.concat([dropped, unique], axis=1)
    return result

def multivariate_regression(df: pd.DataFrame, model='linear'):
    def apply_regressions(df: pd.DataFrame):
        unstacked = df.unstack([sym.category, sym.label])
        unstacked = drop_column_level_duplication(unstacked, 'quality', 0)
        results = regressions(unstacked, quality_column_name='quality', model=model)
        return results
    result = apply_regressions(df)
    result.index = [
        (idx[1] if isinstance(idx, tuple) else idx)
        for idx in result.index.values
    ]
    return result.round(5)

from collections import namedtuple

@to_file
def incremental_regression(
        df: pd.DataFrame,
        categories,
        model='linear',
        exclude_quality=True,
        beam=1,
        select='backward'
):
    data_points = set(df.index.get_level_values('dialogue'))
    num_data_points = len(data_points)
    adjust = lambda r2, f: 1 - (1 - r2) * ((num_data_points - 1) / (num_data_points - f))
    Step: type = namedtuple('Step', ('r2', 'p', 'feature'))
    class Path(list):
        def metric(self):
            # if len(self) == 0: return 0
            # else: return self[-1].llr if len(self) == 1 else self[-1].llr / self[-2].llr
            return self.r2
        @property
        def r2(self):
            return adjust(self[-1].r2, len(self)) if self else 0
        # @property
        # def adj_r2(self):
        #     return adjust(self.r2, len(self))
        @property
        def p(self): return self[-1].p if self else 1
        @property
        def features(self): return {x.feature for x in self}
    r2_name = stat.r2 if model=='linear' else stat.mcfad_r2
    p_name = stat.p_of_f_test if model=='linear' else stat.p_of_llr_test
    frontier = [Path()]
    feature_pool = {
        x[:2] for x in df.index.values
        if (not (exclude_quality and scale.quality in x))
        and x[0] in categories
    }
    for _ in feature_pool:
        new_frontier = []
        for path in frontier:
            for candidate in feature_pool - path.features:
                if select == 'forward':
                    candidate_features = path.features | {candidate}
                elif select == 'backward':
                    candidate_features = feature_pool - path.features
                else:
                    raise ValueError('param select must be one of {"forward", "backward"}')
                row_mask = [
                    x[:2] in candidate_features
                    and (not (exclude_quality and scale.quality in x))
                    and x[0] in categories
                    for x in df.index.values
                ]
                candidate_df = df.loc[row_mask, :]
                candidate_results = multivariate_regression(candidate_df, model=model)
                r2 = candidate_results[r2_name].item()
                p = candidate_results[p_name]
                new_frontier.append(Path([*path, Step(r2, p, candidate)]))
        frontier = sorted(new_frontier, key=lambda x: x.metric(), reverse=True)[:beam]
    result = {step.feature: {'Adjusted R-Squared': step.r2, p_name: step.p} for step in frontier[0]}
    return pd.DataFrame(result.values(), result)


ldq, ltq, icq, idq = all_dialogue_metrics(data)
regs = incremental_regression(
    ldq, (category.likert_turn, category.behavior), beam=1,
    load='results/dialogue_incremental_regressions'
)
regs

Adjusted R-Squared  P value of F-test
likert turn consistent                            0.17799            0.00000
behavior    empathetic                            0.17776            0.00000
            uninterpretable                       0.16896            0.00000
likert turn emotional                             0.16587            0.00000
behavior    ignore                                0.16495            0.00000
            preference info                       0.16420            0.00000
likert turn informative                           0.16244            0.00000
            grammatical                           0.16110            0.00000
behavior    antisocial                            0.15754            0.00000
likert turn relevant                              0.15748            0.00000
behavior    redundant                             0.15261            0.00000
            correct fact                          0.14719            0.00000
            topic switch                          0.14207            0.00000
            commonsense contradiction             0.13828            0.00000
            life info                             0.13499            0.00000
likert turn engaging                              0.13445            0.00000
behavior    incorrect fact                        0.13114            0.00000
likert turn proactive                             0.12112            0.00000
behavior    follow up                             0.11408            0.00000
            self contradiction                    0.11289            0.00000
            partner contradiction                 0.09110            0.00000
            lack of empathy                       0.06708            0.00000
            irrelevant                            0.04943            0.00001

In [54]:
behavior_regs = incremental_regression(
    ldq, (category.behavior,), beam=10,
    load='results/behavior_incremental_regressions'
)
behavior_regs

Adjusted R-Squared  P value of F-test
behavior antisocial                            0.15964                0.0
         ignore                                0.15955                0.0
         life info                             0.15907                0.0
         preference info                       0.15813                0.0
         commonsense contradiction             0.15663                0.0
         uninterpretable                       0.15466                0.0
         correct fact                          0.15197                0.0
         topic switch                          0.14809                0.0
         follow up                             0.14527                0.0
         redundant                             0.14366                0.0
         lack of empathy                       0.13925                0.0
         incorrect fact                        0.13547                0.0
         self contradiction                    0.12608                0.0
         irrelevant                            0.11049                0.0
         empathetic                            0.09489                0.0
         partner contradiction                 0.06043                0.0

In [55]:
behavior_regs = incremental_regression(
    idq, (category.behavior,), beam=10,
    load='results/behavior_incremental_regressions_interactive'
)
behavior_regs

Adjusted R-Squared  P value of F-test
behavior antisocial                            0.26030                0.0
         life info                             0.26013                0.0
         topic switch                          0.25980                0.0
         preference info                       0.25966                0.0
         incorrect fact                        0.25940                0.0
         partner contradiction                 0.25912                0.0
         uninterpretable                       0.25831                0.0
         follow up                             0.25716                0.0
         correct fact                          0.25588                0.0
         ignore                                0.25222                0.0
         commonsense contradiction             0.24564                0.0
         empathetic                            0.23713                0.0
         redundant                             0.22221                0.0
         irrelevant                            0.18954                0.0
         self contradiction                    0.14296                0.0
         lack of empathy                       0.10348                0.0

In [56]:
behavior_regs_comp = incremental_regression(
    icq, (category.behavior,), beam=10,
    load='results/behavior_incremental_regressions_comparative'
)
behavior_regs_comp

Adjusted R-Squared  P value of F-test
behavior antisocial                            0.18339            0.00221
         ignore                                0.18338            0.00129
         topic switch                          0.18322            0.00074
         preference info                       0.18310            0.00040
         partner contradiction                 0.18230            0.00022
         correct fact                          0.18098            0.00012
         irrelevant                            0.17869            0.00007
         empathetic                            0.17866            0.00003
         commonsense contradiction             0.17335            0.00002
         life info                             0.16845            0.00001
         uninterpretable                       0.16153            0.00001
         follow up                             0.15323            0.00001
         redundant                             0.14371            0.00001
         incorrect fact                        0.12315            0.00002
         self contradiction                    0.10889            0.00002
         lack of empathy                       0.05880            0.00070

In [57]:
likert_turn_regs = incremental_regression(
    ldq, (category.likert_turn,), beam=10,
    load='results/likert_turn_incremental_regressions'
)
likert_turn_regs

Adjusted R-Squared  P value of F-test
likert turn grammatical             0.08566            0.00001
            informative             0.08508            0.00000
            engaging                0.08285            0.00000
            emotional               0.07753            0.00000
            consistent              0.06889            0.00000
            proactive               0.05786            0.00001
            relevant                0.03776            0.00009

In [58]:
likert_turn_regs = incremental_regression(
    idq, (category.likert_turn,), beam=10,
    load='results/likert_turn_incremental_regressions_interactive'
)
likert_turn_regs

Adjusted R-Squared  P value of F-test
likert turn informative             0.14708                0.0
            grammatical             0.14651                0.0
            emotional               0.14410                0.0
            consistent              0.13808                0.0
            relevant                0.12962                0.0
            proactive               0.10285                0.0
            engaging                0.06687                0.0

In [59]:
likert_turn_regs_comp = incremental_regression(
    icq, (category.likert_turn,), beam=10,
    load='results/likert_turn_incremental_regressions_comparative'
)
likert_turn_regs_comp

Adjusted R-Squared  P value of F-test
likert turn emotional               0.19089                0.0
            engaging                0.19059                0.0
            grammatical             0.19002                0.0
            relevant                0.18330                0.0
            informative             0.17103                0.0
            consistent              0.14697                0.0
            proactive               0.12035                0.0

In [60]:
likert_dialogue_regs = incremental_regression(
    ldq, (category.likert_dialogue,), beam=10,
    load='results/likert_dialogue_incremental_regressions'
)
likert_dialogue_regs

Adjusted R-Squared  P value of F-test
likert dialogue emotional               0.58426                0.0
                informative             0.57832                0.0
                grammatical             0.56439                0.0
                relevant                0.54517                0.0
                proactive               0.51636                0.0
                consistent              0.48673                0.0
                engaging                0.38103                0.0

In [61]:
likert_dialogue_regs = incremental_regression(
    idq, (category.likert_dialogue,), beam=10,
    load='results/likert_dialogue_incremental_regressions_interactive'
)
likert_dialogue_regs

Adjusted R-Squared  P value of F-test
likert dialogue informative             0.12078                0.0
                consistent              0.12078                0.0
                grammatical             0.12052                0.0
                engaging                0.12010                0.0
                proactive               0.11942                0.0
                emotional               0.10604                0.0
                relevant                0.08183                0.0

In [62]:
likert_dialogue_regs_comp = incremental_regression(
    icq, (category.likert_dialogue,), beam=10,
    load='results/likert_dialogue_incremental_regressions_comparative'
)
likert_dialogue_regs_comp

Adjusted R-Squared  P value of F-test
likert dialogue relevant                0.16572            0.00002
                grammatical             0.16572            0.00001
                informative             0.16470            0.00000
                proactive               0.16335            0.00000
                consistent              0.15861            0.00000
                emotional               0.15367            0.00000
                engaging                0.14639            0.00000

In [63]:
comparative_dialogue_regs_comp = incremental_regression(
    icq, (category.comparative,), beam=10,
    load='results/comparative_incremental_regressions_comparative'
)
comparative_dialogue_regs_comp

Adjusted R-Squared  P value of F-test
comparative proactive               0.05369            0.17270
            consistent              0.05298            0.11737
            relevant                0.05138            0.07843
            grammatical             0.04877            0.05184
            informative             0.04748            0.02712
            emotional               0.04205            0.01725
            engaging                0.03525            0.00912

## Table for Paper

In [64]:
def add_delta(df):
    df['R-Squared'] = (df['Adjusted R-Squared']*100)
    df['delta'] = df['R-Squared'].diff()
    df['delta'] = df['delta'].map('{:.4f}'.format)
    df['R-Squared'] = df['R-Squared'].map('{:.4f}'.format)
    df['R-Squared delta'] = df['R-Squared'] + ' (' + df['delta'] + ')'

final_behavior_regs = behavior_regs.reset_index().rename({'level_1': 'ABC-Eval'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_behavior_regs)

final_likert_turn_regs = likert_turn_regs.reset_index().rename({'level_1': 'Likert Turn'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_likert_turn_regs)

final_likert_dialogue_regs = likert_dialogue_regs.reset_index().rename({'level_1': 'Likert Dialogue'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_likert_dialogue_regs)

final_behavior_regs

,ABC-Eval,Adjusted R-Squared,R-Squared,delta,R-Squared delta
0,antisocial,0.26030,26.0300,nan,26.0300 (nan)
1,life info,0.26013,26.0130,-0.0170,26.0130 (-0.0170)
2,topic switch,0.25980,25.9800,-0.0330,25.9800 (-0.0330)
3,preference info,0.25966,25.9660,-0.0140,25.9660 (-0.0140)
4,incorrect fact,0.25940,25.9400,-0.0260,25.9400 (-0.0260)
5,partner contradiction,0.25912,25.9120,-0.0280,25.9120 (-0.0280)
6,uninterpretable,0.25831,25.8310,-0.0810,25.8310 (-0.0810)
7,follow up,0.25716,25.7160,-0.1150,25.7160 (-0.1150)
8,correct fact,0.25588,25.5880,-0.1280,25.5880 (-0.1280)
9,ignore,0.25222,25.2220,-0.3660,25.2220 (-0.3660)


In [65]:
combined = pd.concat(
    [
        final_behavior_regs[['ABC-Eval', 'R-Squared delta']],
        final_likert_turn_regs[['Likert Turn', 'R-Squared delta']],
        final_likert_dialogue_regs[['Likert Dialogue', 'R-Squared delta']]
    ],
    axis=1)

combined.to_csv('results/paper/incremental_validity.csv', index=False)
combined

,ABC-Eval,R-Squared delta,Likert Turn,R-Squared delta,Likert Dialogue,R-Squared delta
0,antisocial,26.0300 (nan),informative,14.7080 (nan),informative,12.0780 (nan)
1,life info,26.0130 (-0.0170),grammatical,14.6510 (-0.0570),consistent,12.0780 (0.0000)
2,topic switch,25.9800 (-0.0330),emotional,14.4100 (-0.2410),grammatical,12.0520 (-0.0260)
3,preference info,25.9660 (-0.0140),consistent,13.8080 (-0.6020),engaging,12.0100 (-0.0420)
4,incorrect fact,25.9400 (-0.0260),relevant,12.9620 (-0.8460),proactive,11.9420 (-0.0680)
5,partner contradiction,25.9120 (-0.0280),proactive,10.2850 (-2.6770),emotional,10.6040 (-1.3380)
6,uninterpretable,25.8310 (-0.0810),engaging,6.6870 (-3.5980),relevant,8.1830 (-2.4210)
7,follow up,25.7160 (-0.1150),NaN,NaN,NaN,NaN
8,correct fact,25.5880 (-0.1280),NaN,NaN,NaN,NaN
9,ignore,25.2220 (-0.3660),NaN,NaN,NaN,NaN
